Here’s a restructured and concise version of your content:

---

## YOLO Vehicle Registration Plate Detection

**Object Detection Overview:**
There are three primary object detection techniques:
- **R-CNN Family:** R-CNN, Fast R-CNN, and Faster R-CNN
- **SSDs (Single Shot Detectors)**
- **YOLO (You Only Look Once)**

### R-CNN:
R-CNN was one of the first deep learning-based object detection techniques, introduced in the paper *[Rich Feature Hierarchies for Accurate Object Detection and Semantic Segmentation](https://arxiv.org/abs/1311.2524)* in 2013. R-CNN involves two stages: selective search for bounding boxes and classification of those boxes.

While accurate, it was slow, leading to the development of:
- **Fast R-CNN**: Improved speed but still used selective search.
- **Faster R-CNN**: Introduced the Region Proposal Network (RPN), eliminating selective search and improving both speed and accuracy.

Despite its improvements, R-CNN still had limitations in speed, achieving only 5 FPS on a GPU.

### One-Stage Detector:
To improve speed, **Single Shot Detectors (SSDs)** and **YOLO** emerged, treating object detection as a regression problem. These methods detect objects and predict bounding boxes simultaneously, making them faster but less accurate than two-stage detectors like R-CNN.

### YOLO:
**YOLO (You Only Look Once)** was introduced in the paper *[You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/abs/1506.02640)* in 2015. YOLO achieved 45 FPS on a GPU, significantly faster than R-CNN. Over the years, YOLO has evolved with versions like:
- **YOLOv2**: *[YOLO9000: Better, Faster, Stronger](https://arxiv.org/abs/1612.08242)*, capable of detecting over 9,000 objects.
- **YOLOv3**: *[YOLOv3: An Incremental Improvement](https://arxiv.org/abs/1804.02767)*, larger and more accurate than its predecessors.

### YOLO Architecture:
YOLO consists of 24 convolutional layers followed by two fully connected layers. Alternating 1×1 convolutional layers reduce the feature space from previous layers. The convolutional layers are pre-trained on ImageNet classification at a lower resolution (224×224) and then doubled for object detection.

![YOLO Architecture](https://tekworld.org/wp-content/uploads/2019/01/Screen-Shot-2019-01-24-at-9.54.45-PM-1024x447.png)  
Source: *[YOLO Paper](https://arxiv.org/pdf/1506.02640.pdf)*

---

Now, let's implement a simple YOLO-based vehicle registration plate detection program.



In [ ]:
!pip install opencv-contrib-python
!pip install opencv-python
!pip install pytesseract
!pip install ultralytics

In [ ]:
import cv2
import numpy as np
import pytesseract
from glob import glob
import os
import random
from ultralytics import YOLO

In [ ]:
# !wget "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"

In [ ]:
# !wget "https://pjreddie.com/media/files/yolov3.weights"
# !wget "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"

In [ ]:
# Load YOLO model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

In [ ]:
# Load image
img = cv2.imread("car1.jpg")


In [ ]:
# Preprocess image
img = cv2.resize(img, (416, 416))
img = img / 255.0


In [ ]:
# read in video paths
videos = glob('*.mp4')
print(videos)

In [ ]:
# pick pre-trained model
model_pretrained = YOLO('yolov8n.pt')

In [ ]:
# read video by index
video = cv2.VideoCapture(videos[0])

# get video dims
frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('./outputs/uk_dash_2.avi', fourcc, 20.0, size)

# read frames
ret = True

while ret:
    ret, frame = video.read()

    if ret:
        # detect & track objects
        results = model_pretrained.track(frame, persist=True)

        # plot results
        composed = results[0].plot()

        # save video
        out.write(composed)

out.release()
video.release()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

import zipfile
import os

# Path to the ZIP file
zip_file_path = 'lpr_yolov8.zip'

# Directory to extract the contents
extract_dir = 'lpr_yolov8'

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("ZIP file extracted to:", extract_dir)

In [ ]:
# unzip downloaded dataset to `./datasets`
dataset = '/content/dataset/lpr_yolov8/data.yaml'

# load a model
# backbone = YOLO("yolov8n.yaml")  # build a new model from scratch
backbone = YOLO("yolov8n.pt")  # load a pre-trained model (recommended for training)

In [ ]:
# Use the model
# results = backbone.train(data=dataset, epochs=20)  # train the model

In [ ]:
!gdown "https://drive.google.com/uc?id=1dIyJooVaowaNUj0R1Q-HUnu-utiGsEj8&confirm=t"

In [ ]:
# regular pre-trained yolov8 model for car recognition
coco_model = YOLO('yolov8s.pt')
# yolov8 model trained to detect number plates
np_model = YOLO('best.pt')

In [ ]:
# # car detection

# # read video by index
# video = cv2.VideoCapture(videos[1])

# ret = True
# frame_number = -1
# # all vehicle class IDs from the COCO dataset (car, motorbike, truck) https://docs.ultralytics.com/datasets/detect/coco/#dataset-yaml
# vehicles = [2,3,5]
# vehicle_bounding_boxes = []

# # read the 10 first frames
# while ret:
#     frame_number += 1
#     ret, frame = video.read()

#     if ret and frame_number < 10:
#         # use track() to identify instances and track them frame by frame
#         detections = coco_model.track(frame, persist=True)[0]
#         # save cropped detections
#         # detections.save_crop('outputs')
#         # print nodel predictions for debugging
#         # print(results)

#         for detection in detections.boxes.data.tolist():
#             # print detection bounding boxes for debugging
#             # print(detection)
#             x1, y1, x2, y2, track_id, score, class_id = detection
#             # I am only interested in class IDs that belong to vehicles
#             if int(class_id) in vehicles and score > 0.5:
#                 vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])

# # print found bounding boxes for debugging
# print(vehicle_bounding_boxes)
# video.release()

In [ ]:
# License plate detection

# read video by index
video = cv2.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 10:

        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    # debugging check if bbox lines up with detected vehicles (should be identical to save_crops() above
                    # cv.imwrite(str(track_id) + '.jpg', roi)

                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # check every bounding box for a license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # verify detections
                        print(license_plate, 'track_id: ' + str(bbox[4]))
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        cv2.imwrite(str(track_id) + '.jpg', plate)

video.release()

In [ ]:
from google.colab.patches import cv2_imshow

# License plate detection

# Define the video file name (ensure the MP4 file is in the same directory as the script)
video_file = 'traffic.mp4'  # Replace with your actual MP4 file name

# Get the absolute path of the video file
# Get the absolute path of the video file
video_path = os.path.join(os.getcwd(), video_file)

# Read video from the file
video = cv2.VideoCapture(video_path)

ret = True
frame_number = -1
vehicles = [2, 3, 5]

# Read the first 10 frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 10:
        # Display the current video frame using cv2_imshow (for Colab)
        cv2_imshow(frame)

        # Vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]

                    # Display the vehicle region of interest (ROI)
                    cv2_imshow(roi)

                    # License plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # Verify detections
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]

                        # Display the detected license plate
                        cv2_imshow(plate)

                        # Save the detected license plate
                        cv2.imwrite(str(track_id) + '.jpg', plate)

# Release video capture
video.release()

